## ch03. Our First Python Forensics App

## 명명 규칙 및 기타 고려사항

- 명명을 해놨는데 왜 죄다 PEP8을 어기는거냐?
- 그래서 이 명명법이 싫지만 책을 따라하는 입장에서는 그대로 따라하는게 가장 머리가 안 아프다. 
- PEP8을 준수해야하지 이 책에 나와있는 코드 명명법을 준수하지 말 것

## 단방향 파일 시스템 해싱

### 목적

1. 포렌식 수라르 위한 도구 및 응용 프로그램을 유용하게 구축
2. 이 책 전반에 걸쳐 장래의 응용 프로그램 및 재사용 방법에 따라 개별 모듈을 개발
3. 파이썬 포렌식 응용 프로그램을 구축하기 위한 신뢰할 수 있는 방법론을 개발
4. 언어에 대한 고급 기능 소개

### 단방향 해싱 알고리즘의 기본 특성

1. 단방향 해싱 알고리즘은 비밀번호, 파일, 하드 드라이브의 이미지, 고체 상태 드라이브의 이미지, 네트워크 패킷, 디지털 기록으로부터 1과 0, 또는 기본적으로 연속적인 디지털이 입력되는 Binary 데이터에 대한 스트림을 취급
2. 이 알고리즘은 입력으로부터 받은 이진 데이터를 압축한 표현인, 메시지 다이제스트를 생성
3. 다이제스트로 다이제스트를 생성하는 바이너리 입력을 결정하는 것은 실행 불가능. 즉 생성된 다이제스트 바이너리 데이터의 스트림을 복구하기 위해서 다이제스트를 사용하여 되돌릴 수 없음
4. 주어진 메시지 다이제스트를 생성할 새로운 이진 입력 데이터를 생성하는 것은 불가능
5. 이진 입력 데이터에 대한 단일 비트를 바꾸면 고유한 메시지 다이제스트를 생성할 것
6. 마지막으로 동일한 다이제스트를 산출하는 이진 데이터에 대한, 2개의 고유한 스트림을 찾는 것은 불가능


- [해시 함수 \- 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%ED%95%B4%EC%8B%9C_%ED%95%A8%EC%88%98)
- [Hash function \- Wikipedia, the free encyclopedia](https://en.wikipedia.org/wiki/Hash_function)
- [해시함수 \- KISA](http://seed.kisa.or.kr/iwt/ko/intro/EgovHashFunction.do): KISA에서 해쉬 함수를 약간 쉽게 설명?
- [암호화 해시 함수 \- 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EC%95%94%ED%98%B8%ED%99%94_%ED%95%B4%EC%8B%9C_%ED%95%A8%EC%88%98)
- [중국 연구팀이 SHA-1 해독에 새로운 진전을 이룩(2005년 버전)](http://egloos.zum.com/sonnet/v/2657614)
- [SHA1 + salt로 패스워드 보안 이슈 회피가 가능한가? \- Mimul's Developer World](http://www.mimul.com/pebble/default/2013/02/18/1361189648941.html)
- [암호 알고리즘 \- 나무위키](https://namu.wiki/w/%EC%95%94%ED%98%B8%20%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98)

#### 단방향 해시 알고리즘의 장단점은?

- MD5 알고리즘의 공격 및 충돌에 대한 가능성이 증가되면서 SHA-2(256비트와 512 비트가 가장 일반적인 크기)로 이동
- SHA-3으로 옮겨가는게 이제 대세가 될 것임

#### 포렌식의 단방향 해시 알고리즘에 대한 최고의 활용 사례?

- 증거 보존
  - Hash가 일치한다면 증거가 변경되지 않음을 입증하는 것
- 검색
  - 아동 포르노 파일들을 상습적으로 수집하여 소장하공 씨는 것
  - 각 파일에 대해서 해시 값이 계산될 수 있음
  - 불법 거래의 존재에 대해서 정밀 검사
  - 불법 거래 해시 값(아동 포르노 수집으로 인한 결과)
- 블랙 리스트
  - 유해한 해시 파일에 대한 목록을 작성하는 것
  - 악의적인 코드
  - 사이버 무기 파일
  - 특허 문서의 해시와 일치할 수도 있음
- 화이트 리스트
  - 양성으로 알려진 해시값(운영체제 또는 실행 가능한 응용 프로그램, 공급업체에서 제공한 동적 라이브러리 또는 신뢰할 수 있는 알려진 응용 프로그램 다운로드 파일)에 대한 목록 작성
- 변화 감지
  - 웹 사이트, 라우터, 방화벽 구성, 운영체제 설치의 악의적인 변경에 대비

## 기본 요구사항

번호 | 요구사항 명 | 설명
--- | --- | ---
000 | 개요 | 여러분이 찾고 있는 기본 기능은 정의된 시작점[예를 들어 c:\ 또는 /etc)에 있는 파일 시스템을 안내하는 포렌식 응용 프로그램이다. 모든 파일에 대한 단방향 해시 값을 생성한다.
001 | 이식성 | 응용 프로그램은 윈도우와 리눅스 운영체제를 지원해야 한다. 일반적인 지침에 따라 검증은 윈도우 7, 윈도우 8, 그리고 우분투 12.04 LTS 운영환경에서 수행된다.
002 | 주요 기능 | 단방향 해시 생성 외에, 응용 프로그램은 해시된 각 파일과 관련된 시스템 메타데이터를 수집해야 한다. 예를 들면 최소한, 파일 속성, 파일 이름, 파일 경로가 있다.
003 | 주요 걸과 | 응용 프로그램은 유연성을 제공하는 표준 출력 파일 형식으로 결과를 제공해야 한다.
004 | 알고리즘 선택 | 응용 프로그램은 사용되는 단방향 해시 알고리즘을 지정할 때 넓은 다양성을 고려해야 한다.
005 | 오류 처리 | 응용 프로그램은 오류 처리를 지원해야만 하고 모든 수행 동작에 대해 기록해야 한다. 이것은 텍스트 설명과 날짜와 시간의 기록을 포함할 것이다.

### 설계 고려사항

- 표준 라이브러리의 내장 함수를 많이 활용
- 중요한 단계이자 재미있는 부분: 프로그램 이름을 붙이는 것
- 파일 시스템 해싱의 약자 p-fish

### 어떤 모듈을 사용할 것인지?

1. 사용자 입력을 위한 argparse
2. 파일 시스템을 다루기 위한 os
3. 단방향 해쉴르 위한 hashlib
4. 결과 출력(다른 임의의 출력은 이후에 추가될 수 있다)을 위한 cvs
5. 이벤트 및 오류에 대한 logging
6. time, sysm, stat와 같이 다방면에 걸쳐 유용한 모듈들

### 이 책의 좋은점

- 요구사항을 만들고
- 설계시 고려사항을 적고
- 라이브러리 선택한 이유를 적고
- 일련의 프로그램 만들기 위한 순서를 알려주기 때문에 나중에 나도 똑같이 따라해 볼 수 있는 여건들을 알려주어서 정말 좋다.

#### 표준 라이브러리 매핑

번호 | 설계 고려사항 | 라이브러리 선택
--- | --- | ---
사용자 입력(000, 003, 004) | 이러한 각각의 요건들은 과업을 달성하기 위해서 사용자로부터의 입력을 필요로 한다. 예를 들어 000은 시작하는 디렉토리 경로를 사용자가 지정할 것을 요구한다. 003은 사용자가 적합한 출력 형식을 지정해야 한다. 004는 해시 알고리즘을 지정할 것을 요구한다. 예외 처리 또는 기본 설정에 대한 자세한 사항은 정의되어야 한다(허용된 경우). | 첫 번째 프로그램을 위해서 사용자 입력을 얻기 위한 명령줄(command line) 매개변수를 사용하기로 결정했다. 이러한 설계를 기반으로 파이썬 라이브러리 모듈 argparse를 활용할 것을 결정했다.
파일 시스템 조작(000, 001) | 이 기능은 특정한 출발점에서 시작하는 디렉토리 구조를 횡단할 수 있는 프로그램을 요구한다. 또한 윈도우와 리눅스 플랫폼 모두에서 작동해야 한다. | 표준 라이브러리에서 OS 모듈은 파일 시스템을 탐색할 수 있는 기능과 교차 사용이 가능한 플랫폼 호환성을 제공하는 추상화를 제공하는 주요 메소드를 공급한다. 이 모듈은 파일과 관련된 메타 데이터에 대한 접근을 제공하는 Cross Platform 기능을 포함하고 있다.
메타 데이터 수집(003) | 디렉토리 경로, 파일 이름, 소유자, 작성/수정/접근 시간, 권한, 읽기 전용, 숨김과 같은 속성, 시스템 또는 기록(archive)을 수집할 것을 요구한다. | 표준 라이브러리에서 OS 모듈은 파일 시스템을 탐색할 수 있는 기능과 교차 사용이 가능한 플랫폼 호환성을 제공하는 추상화를 제공하는 주요 메소드를 공급한다. 이 모듈은 파일과 관련된 메타 데이터에 대한 접근을 제공하는 Cross Platform 기능을 포함하고 있다.
파일 해시(000) | 사용자가 선택할 수 있는 해시 알고리즘에 유연성을 제공해야 한다. MD5와 몇 가지 다양한 SHA와 같이 가장 대중적인 알고리즘을 지원하기로 결정했다. | 표준 라이브러리 hashlib 모듈은 단방향 해시 값을 생성하는 기능을 제공한다. 라이브러리는 "MD5", "SHA1", "SHA224", "SHA256", "SHA384", "SHA512"와 같은 일반적인 해시 알고리즘을 지원한다. 이것은 사용자에 대한 선택의 충분한 set을 제공한다.
결과 출력 | 이러한 요구사항을 충족하기 위해서 유연성을 가진 포맷을 제공하려면 프로그램 출력을 구조화할 수 있어야 한다. | 표준 라이브러리는 활용할 수 있는 다양한 옵션을 제공한다. 예를 들어 csv 모듈은 쉼표(,)로 구분된 값 출력을 생성할 수 있는 기능을 제공한다. 반면에 json(Java Object Notation) 모듈은 JSON 객체에 대한 암호기 및 복호기를 제공하고, XML 모듈은 XML 출력을 작성하기 위해 활용될 수 있다.
로깅 및 오류 처리 | 우리는 파일시스템에 대해 탐색하는 동안 발생할 수 있는 오류를 예상해야 한다. 예를 들면, 특정 파일에 접근하지 못하거나 특정 파일이 고아가 되는 경우가 있다. 또 운영시스템 또는 다른 응용 프로그램에 의해 잠겨진 특정 파일이 있을 수 있다. 이러한 오류 조건들을 처리하고 주목할만한 사건들을 기록해야 한다. 예를 들어 수사관, 배치, 날짜와 시간, 그리고 탐색중인 시스템에 속하는 정보를 기록해야 한다. | 파이썬 표준 라이브러리는 처리중에 발생하는 모든 이벤트 또는 오류를 보고하기 위해서 활용할 수 있는 로깅 기능을 포함하고 있다.

### 프로그램 구조

#### Main 함수

- 파이썬 로거 설정
- 시작 및 완료 메시지 표시
- 시간에 대한 기록
- 명령줄 파서를 호출한 다음 WalkPath 함수 실행
- WalkPath가 완료되면 Main은 완료를 기록하고 사용자와 로그에 대한 종료 메시지를 표시할 것

#### ParseCommandLine

- p-fish에 대한 원활한 작동을 제공하기 위해, 사용자 입력 값의 문장구조 분석뿐만 아니라 유효성 검증을 위해 parseCommandLine을 활용
- 일단 WalkPath, HashFile, CSVWrite 등의 프로그램 기능에 밀접한 정보를 완성하면, 파서로부터 생성된 값에서 사용할 수 있음
- 예를 들어 해시 타입은 사용자에 의해 지정되기 때문에, 이 값은 HashFile에서 사용할 수 있어야 함
- 마찬가지로 CSVWriter는 pfish 보고서 결과를 작성하기 위한 경로를 필요로 하고 WalkPath는 시작하기 위한 시작점 또는 rootPath를 요구한다.

#### WalkPath 함수

- 디렉토리 경로에 대한 루트에서 시작해야 하고 모든 디렉토리와 파일을 통과함
- 유효한 각각의 파일에 대해 단방향 해시 연산을 수행하기 위해서 HashFile 함수를 호출한다.
- 모든 파일이 처리된 후에 WalkPath는 성공적으로 처리된 파일의 수에 따라 제어권을 Main에 반환한다.

#### HashFile 함수

- 열고, 읽고, 해시 연산 및 의심스러운 파일에 대한 메타데이터를 가져온다.
- 각 파일에 대해 데이터의 행은 p-fish 보고서에 포함될 CSVWriter로 전송
- 파일이 처리가 되면 HashFile은 다음 파일을 인출하기 위해서 제어권을 WalkPath에 반환

#### CSVWriter(클래스)

- 클래스 및 객체의 작동이나 사용법에 대한 시범을 멋지게 설정
- csv 모듈은 초기화 할 수 있는 "작가"를 필요로 함
- 예를 들어, 고정된 열(column)의 집합(set)으로 구성된 머리글(header) 행을 가지는 csv 파일을 결과로 하기를 원함
- 각 행을 채운 데이터를 포함
- 마지막으로 일단 프로그램이 처리되면 csv 보고서를 결과로 하는 모든 파일들을 닫아야 함

#### 로거

- 내장된 표준 라이브러리 logger는 p-fish와 관련된 로그 파일에 메시지를 기록할 수 있는 기능
- 정보 메시지, 경고 메시지, 오류 메시지 작성 가능
- 포렌식 응용 프로그램을 위한 것이기 때문에 프로그램의 동작을 기록하는 것은 매우 중요
- 코드 내에 추가적인 이벤트를 기록하기 위해서 프로그램을 확장할 수 있고 \_pfish 함수의 어디에라도 추가할 수 있음

## Code Walk-Through

### 주요 부분 분석하기 - 코드 워크스로

In [3]:
# coding: utf-8
import logging
import time
import sys
import _pfish


if __name__ == '__main__':
    PFISH_VERSION = '1.0'
    # 로깅 설정
    logging.basicConfig(filename='pFishLog.log', 
                        level=logging.DEBUG,
                        format='%(asctime)s %(message)s')
    
    # 명령줄 인수를 처리한다.
    _pfish.ParseCommandLine()
    
    # 시작하는 시간을 기록한다.
    startTime = time.time()
    
    # 로그에 스캔 시작 메시지를 게시한다.
    logging.info('Welcome to p-fish %s... New Scan Started' % (PFISH_VERSION))
    _pfish.DisplayMessage('Welcome to p-fish... %s' % (PFISH_VERSION))
    
    # 시스템에 관여하는 일부 정보를 기록한다.
    logging.info('System: ' + sys.platform)
    logging.info('Version: ' + sys.version)
    
    # 파일 시스템 디렉토리 및 해시 파일들을 횡단함
    filesProcessed = _pfish.WalkPath()
    
    # 종료 시간을 기록하고 지속 시간을 계산함
    endTime = time.time()
    duration = endTime - startTime
    
    logging.info('Files Processed: ' + str(filesProcessed))
    logging.info('Elapsed Time: ' + str(duration) + 'seconds')
    
    logging.info('Program Terminated Normally')
    
    _pfish.DisplayMessage('Program End')

ImportError: No module named _pfish

### ParseCommandLine 함수

1. 첫번째 응용 프로그램이 명령줄 프로그램이 될 것
2. 프로그램의 동작을 능숙하게 다루도록 사용자에게 몇 가지 옵션을 제공하기로 한다. 이것은 명령줄 옵션에 대한 설계와 구현을 주도한다.

옵션 | 설명 | 참고
--- | --- | ---
-v | Verbose, 이 옵션은 지정된 다음에 DisplayMessage() 함수를 호출하는 경우, 표준 입출력 장치에 표시되고, 그렇지 않으면 프로그램이 자동으로 실행됨 | 
--MD5<br />--SHA251<br />--SHA512 | 해시 타입 선택, 사용자가 사용하고자 하는 단방향 해시 알고리즘을 지정해야 한다. | 선택은 상호 배타적이며 적어도 하나를 선택해야 한다. 그렇지 않으면 프로그램이 중단될 것
-d | rootPath, 사용자가 시작 또는 루트 경로를 지정할 수 있다 | 디렉토리가 존재해야 하고, 읽을 수 있어야 한다. 그렇지 않으면 프로그램이 중단될 것이다.
-r | reportpath, 사용자가 결과 .csv 파일이 기록된 디렉토리를 지정할 수 있다. | 디렉토리가 존재해야 하고 쓸 수 있어야 한다. 그렇지 않으면 프로그램이 중단될 것이다.

- 구문분석 프로그램을 설정하는 방법에 대해 아는 것
- -v는 자세한 설명
- -h는 기봊 제공

In [7]:
import argparse

def ParseCommandLine():
    parser = argparse.ArgumentParser('Python file system hashing... p-fish')
    parser.add_argument('-v', 
                        '--verbose', 
                        help='Allows progress messages to be displayed', 
                        action='store_true')


- 사용자가 생성하고자 하는 특정 해시 타입을 선택하기 위한 인수의 상호 베타적인 그룹을 정의
- required = True는 add_mutually_exclusive_group의 안쪽에 지정한 이후에, argparse는 사용자가 하나의 인수와 적어도 하나를 지정했는지를 확인

In [ ]:
group = parser.add_mutually_exclusive_group(required=True)
group.add_argument('--md5', 
                   help='specifies MD5 algorithm', 
                   action='store_true')
group.add_argument('--sha251', 
                   help='specifies SHA256 algorithm', 
                   action='store_true')
group.add_argument('--sha512', 
                   help='specifies SHA512 algorithm', 
                   action='store_true')

- 행동의 시작점을 지정해야 하고, 보고서가 만들어질 장소가 필요함
- -d: rootPath가 존재하고 읽기 가능한지 확인
- argparse는 디렉토리 유효성 검사를 위한 내장된 기능을 가지고 있지 않기 때문에, ValidateDirectory()와 ValidateDirectoryWritable() 함수를 만듦. 그것들은 거의 동일하고 정의된 표준 라이브러리 운영체제 기능을 사용함

In [ ]:
parser.add_argument('-d', 
                    '--rootPath', 
                    type=ValidateDirectory,
                    required=True,
                    help='Specify the root path for hashing')
parser.add_argument('-r', 
                    '--reportPath',
                    type=ValidateDirectoryWritable,
                    required=True,
                    help='Specify the path for reports and logs will be written')

# 검증된 인수를 보유하는 전역 객체를 생성한다.
# 이러한 전역 객체는 # _pfish.py 모듈 내의 모든 함수에서 사용가능
global gl_args
global gl_hashType

gl_args = parser.parse_args()

- 파서가 성공하면, 해시값을 연산하는 알고리즘을 결정한다.
- 예를 들어, 사용자가 SHA256을 선택한 경우, gl_args.sha256은 True가 되고 MD5 및 SHA512는 False가 됨. 따라서 간단한 if/elif 언어 루틴을 사용하면 선택한 것을 결정할 수 있음

In [ ]:
if gl_args.md5:
    gl_hashType = 'MD5'
elif gl_args.sha256:
    gl_hashType = 'SHA256'
elif gl_args.sha512:
    gl_hashType = 'SHA512'
else:
    gl_hashType = 'Unknown'
    logging.error('Unknown Hash Type Specified')
return DisplayMessage('Command line processed: Successfully')

### Validating DirectoryWritable

- 보고서 및 시작 루트 경로을 확인할 필요가 있다. 
- os.path.isdir과 os.access 메소드를 활용

In [ ]:
def ValidateDirectoryWritable(theDir):
    # path가 디렉토리의 유효성을 ㄱ머사한다.
    if not os.path.isdir(theDir):
        raise argparse.ArgumentTypeError('디렉토리가 존재하지 않습니다.')
    # access가 쓰기 가능한지 확인
    if os.access(theDir, os.W_OK):
        return theDir
    else:
        raise argparse.ArgumentTypeError('Directory is not writable')

### ParseCommandLine Testcase

1. TEST_DIR 대신에 TESTDIR로 루트 디렉토리를 잘못 입력
2. --sha512 매개변수를 -sha512로 잘못 입력
3. --sha512와 --md5로 2개의 해시 타입을 지정
4. 어떤 해시 타입도 지정하지 않음

## WalkPath

- 디렉토리 구조를 횡단하며 각 파일에 대한 HashFile 함수를 호출할 수 있는 WalkPath 함수로 이동

In [ ]:
processCount = 0
errorCount = 0

log.info('Root Path: ' + gl_args.rootPath)

In [9]:
oCVS = _CSVWriter(gl_args.reportPath + 'fileSystemReport.csv', gl_hashType)

# 시작하는 모든 파일을 처리하는 루프를 생성한다
# rootPath에서 모든 하위 디렉토리도 처리된다.
# processed

In [11]:
import os

In [12]:
help(os.walk)

Help on function walk in module os:

walk(top, topdown=True, onerror=None, followlinks=False)
    Directory tree generator.
    
    For each directory in the directory tree rooted at top (including top
    itself, but excluding '.' and '..'), yields a 3-tuple
    
        dirpath, dirnames, filenames
    
    dirpath is a string, the path to the directory.  dirnames is a list of
    the names of the subdirectories in dirpath (excluding '.' and '..').
    filenames is a list of the names of the non-directory files in dirpath.
    Note that the names in the lists are just names, with no path components.
    To get a full path (which begins with top) to a file or directory in
    dirpath, do os.path.join(dirpath, name).
    
    If optional arg 'topdown' is true or not specified, the triple for a
    directory is generated before the triples for any of its subdirectories
    (directories are generated top down).  If topdown is false, the triple
    for a directory is generated after the 

In [ ]:
for dirpath, dirnames, filenames in os.walk(gl_args.rootPath):
    # for는 각 파일에 대해 파일 이름을 획득하고
    # HashFile 함수를 호출한다.
    for f in filenames:
        fname = os.path.join(root, f)
        result = HashFile(fname, f, oCVS)
        
        # 성공하면 ProcessCount가 증가한다.
        if result:
            processCount += 1
        # 성공하지 않으면, ErrorCount가 증가한다.
        else:
            errorCount += 1

In [ ]:
oCVS.writerClose()
return processCount

### HashFile

In [ ]:
def HashFile(theFile, simpleName, o_result):
    

#### Hash 시도 전 검증

1. 경로가 존재하는가?
2. 실재 파일 대신에 연결하는 경로가 있는가?
3. 파일이 실재(고아가 되지 않았다는 것을 확인할 수 있는)하는가?

In [ ]:
# 경로가 유효한지 확인한다.
if os.path.exists(theFile):
    # 경로가 심볼 링크가 아닌지 확인한다.
    if not os.path.islink(theFile):
        # 파일이 실재하는지 확인한다.
        if os.path.isfile(theFile):

In [ ]:
# 맞는 패턴인지는 TestCase를 만들어서 해봐야 한다.
if not os.path.exists(theFile):
    return False
if os.path.islink(theFile):
    return False
if not os.path.isfile(theFile):
    return False

- 이렇게 하면 너무 안쪽으로 들어오기 때문에 [Replace Nested Conditional with Guard Clauses Pattern](http://refactoring.com/catalog/replaceNestedConditionalWithGuardClauses.html) 적용해서 해보자
- [Replace Nested Conditional with Guard Clauses](https://sourcemaking.com/refactoring/replace-nested-conditional-with-guard-clauses)

In [ ]:
try:
    # 파일 읽기 시도
    f = open(theFile, 'rb')
except IOError:
    # 열기를 실패할 경우 오류 보고
    log.warning('Open Failed: ' + theFile)
    return
else:
    try:
        # 파일 읽기 시도
        rd = f.read()
    except IOError:
        # 읽기를 실패할 경우, 파일을 닫고
        # 오류를 보고한다.
        f.close()
        log.warning('Read Failed: ' + theFile)
        return
    else:
        # 파일 열기가 성공하면 이 파일로부터 읽을 수 있다.
        # 파일 상태를 조회하자.
        theFileStats = os.stat(theFile) # 이건 뭐하러 함??
        (mode, ino, dev, nlink, uid, gid, size, atime, mtime, ctime) = os.stat(theFile)
        
        # 사용자에게 진행중을 표시함
        DisplayMessage('Processing File: ' + theFile)
        
        # 파일 크기를 문자열로 변환
        fileSize = str(size)
        
        # 수정/접근/속성변경 시간을 문자열로 변환
        modifiedTime = time.ctime(mtime)
        accessTime = time.ctime(atime)
        createdTime = time.ctime(ctime)
        
        # 소유자, 그룹, 파일 모드를 변환
        ownerID = str(uid)
        groupID = str(gid)
        fileMode = bin(mode)

- 파일을 성공적으로 열고 파일을 읽는 것이 허용되면 파일과 관련된 속성을 추출

### 조각 코드??

- 조각 조각 나눠서 설명할 때는 좋은데..
- 내가 굳이 다시 한 번 써야되나??
- 그냥 Full Source Code만 작성해도 될 것 같다.
- 다 이해했으니까.

### CSVWriter

#### 초기화 메소드

1. csvFile 열기
2. csv.writer 초기화
3. 각 칼럼에 대한 이름과 함께 머리글 행을 기록

#### Full Source Code pfish.py

In [16]:
!mkdir ch03

In [19]:
%%writefile ch03/pfish.py

# coding: utf-8
import logging
import time
import sys
import _pfish


if __name__ == '__main__':
    PFISH_VERSION = '1.0'

    # 로깅을 설정한다.
    logging.basicConfig(filename='pFishLog.log',
                        level=logging.DEBUG,
                        format='%(asctime)s %(message)s')

    # 명령줄 인수 처리
    _pfish.ParseCommandLine()

    # 시작하는 시간 기록
    startTime = time.time()

    # 환영 메시지 기록
    logging.info('')
    logging.info('Welcome to p-fish version {} ... New Scan Started'.format(
            PFISH_VERSION))
    logging.info('')
    _pfish.DisplayMessage('Welcome to p-fish ... version'.format(PFISH_VERSION))

    # 시스템과 관련한 일부 정보 기록
    logging.info('System: {}'.format(sys.platform))
    logging.info('Version: {}'.format(sys.version))

    # 파일 시스템 디렉토리 및 해시 파일을 횡단함
    fileProcessed = _pfish.WalkPath()

    # 종료 시간을 기록하고 기간을 계산
    endTime = time.time()
    duration = endTime - startTime
    logging.info('Files Processed:' + str(fileProcessed))
    logging.info('Elapsed Time: ' + str(duration) + 'seconds')
    logging.info('')
    logging.info('Program Terminated Normally')
    logging.info('')

    _pfish.DisplayMessage('Program End')

Overwriting ch03/pfish.py


In [20]:
%%writefile ch03/_pfish.py

# coding: utf-8
import os
import stat
import time
import hashlib
import argparse
import csv
import logging

log = logging.getLogger('main._pfish')


def ParseCommandLine():
    parser = argparse.ArgumentParser('Python file system hashing... p-fish')

    parser.add_argument('-v',
                        '--verbose',
                        help='allows progress messages to be displayed',
                        action='store_true')

    # 상호 베타적인 선택이 필요한 그룹 설정
    group = parser.add_mutually_exclusive_group(required=True)
    group.add_argument('--md5',
                       help='specifies MD5 algorithm',
                       action='store_true')
    group.add_argument('--sha256',
                       help='specifies SHA256 algorithm',
                       action='store_true')
    group.add_argument('--sha512',
                       help='specifies SHA512 algorithm',
                       action='store_true')

    parser.add_argument('-d',
                        '--rootPath',
                        type=ValidateDirectory,
                        required=True,
                        help='specify the root path for hashing')
    parser.add_argument('-r',
                        '--reportPath',
                        type=ValidateDirectoryWritable,
                        required=True,
                        help='specify the path for reports and logs will be written')

    # 검증된 인수를 보유한 전역 객체를 생성
    # 이 객체는 _pfish.py 모듈 내의 모든 함수에서 사용 가능

    global gl_args
    global gl_hashType

    gl_args = parser.parse_args()

    if gl_args.md5:
        gl_hashType = 'MD5'
    elif gl_args.sha256:
        gl_hashType = 'SHA256'
    elif gl_args.sha512:
        gl_hashType = 'SHA512'
    else:
        gl_hashType = 'UnKnown'
        logging.error('Unknown Hash Type Specified')

    DisplayMessage('Command line processed: Successfully')
    return


def WalkPath():
    processCount = 0
    errorCount = 0

    oCVS = _CSVWriter(os.path.join(gl_args.reportPath, 'fileSystemReport.csv'),
                      gl_hashType)

    # rootPath에서 시작하는 모든 파일을 처리하는 반복문을 만들고,
    # 모든 하위 디렉토리도 처리된다.

    log.info('Root Path: ' + gl_args.rootPath)

    for root, dirs, files in os.walk(gl_args.rootPath):
        # for문은 각 파일에 대한 파일 이름을 획득하고 HashFile 함수를 호출한다.
        for file in files:
            fname = os.path.join(root, file)
            result = HashFile(fname, file, oCVS)

            # 해시 연산이 성공한 경우, ProcessCount가 증가한다.
            if result is True:
                processCount += 1
            # 성공하지 못한 경우, ErrorCount가 증가한다.
            else:
                errorCount += 1

    oCVS.writerClose()
    return(processCount)


def HashFile(theFile, simpleName, o_result):
    """
    Must Refactoring.
    """
    # 경로가 유효한지 확인한다.
    if os.path.exists(theFile):
        # 경로가 심볼 링크가 아닌지 확인한다.
        if not os.path.islink(theFile):
            # 파일이 실재하는지 확인
            if os.path.isfile(theFile):
                try:
                    # 파일 열기 시도
                    f = open(theFile, 'rb')
                except IOError:
                    # 열기에 실패하는 경우, 오류를 보고
                    log.warning('Open Failed: ' + theFile)
                    return
                else:
                    try:
                        # 파일 읽기 시도
                        rd = f.read()
                    except IOError:
                        # 읽기에 실패하는 경우, 파일을 닫고 오류 보고
                        f.close()
                        log.warning('Read Failed: ' + theFile)
                        return
                    else:
                        # 파일 열기가 성공하면 이 파일로부터 읽을 수 있음
                        # 파일의 상태를 조회

                        theFileStats = os.stat(theFile)
                        (mode, ino, dev, nlink, uid, gid, size,
                         atime, mtime, ctime) = os.stat(theFile)

                        # 단순한 파일 이름을 인쇄
                        DisplayMessage('Processing File: ' + theFile)

                        # 파일의 바이트(Byte) 크기를 인쇄
                        fileSize = str(size)

                        # 수정/접근/속성변경 시간을 인쇄
                        modifiedTime = time.ctime(mtime)
                        accessTime = time.ctime(atime)
                        createdTime = time.ctime(ctime)

                        ownerID = str(uid)
                        groupID = str(gid)
                        fileMode = bin(mode)

                        # 파일 해시를 처리

                        if gl_args.md5:
                            # MD5 계산 및 인쇄
                            hashValue = hashlib.md5(rd).hexdigest()
                        elif gl_args.sha256:
                            hashValue = hashlib.sha256(rd).hexdigest()
                        elif gl_args.sha512:
                            hashValue = hashlib.sha512(rd).hexdigest()
                        else:
                            log.error('Hash not Selected')

                            # 파일 처리가 완료되면
                            # 활성 상태의 파일을 닫는다.
                            print('=' * 20)
                            f.close()

                        # 출력 파일에 한 행을 기록함
                        o_result.writeCSVRow(simpleName, theFile, fileSize,
                                             modifiedTime, accessTime, createdTime,
                                             hashValue, ownerID, groupID, mode)
                        return True
            else:
                log.warning('[{} Skipped NOT a File]'.format(repr(simpleName)))
                return False
        else:
            log.warning('[{} Skipped NOT a File]'.format(repr(simpleName)))
            return False
    else:
        log.warning('[{} Path does NOT a exist]'.format(repr(simpleName)))
        return False


def ValidateDirectory(theDir):
    # 디렉토리 경로의 유효성을 검사
    if not os.path.isdir(theDir):
        raise argparse.ArgumentTypeError('Directory does not exist')

    # 경로가 읽기 가능한지 검사
    if os.access(theDir, os.R_OK):
        return theDir
    else:
        raise argparse.ArgumentTypeError('Directory is not readable')


def ValidateDirectoryWritable(theDir):
    # 디렉토리 경로의 유효성 검사
    if not os.path.isdir(theDir):
        raise argparse.ArgumentTypeError('Directory does not exist')

    # 경로가 쓰기 가능한지 검사
    if os.access(theDir, os.W_OK):
        return theDir
    else:
        raise argparse.ArgumentTypeError('Directory is not writable')


def DisplayMessage(msg):
    if gl_args.verbose:
        print(msg)
    return


class _CSVWriter(object):
    def __init__(self, fileName, hashType):
        try:
            # writer 객체를 생성하고 머리글 행을 기록
            self.csvFile = open(fileName, 'wb')
            self.writer = csv.writer(self.csvFile, delimiter=',', quoting=csv.QUOTE_ALL)
            self.writer.writerow(('File', 'Path', 'Size', 'Modified Time', 'Access Time',
            'Created Time', hashType, 'Owner', 'Group', 'Mode'))
        except:
            log.error('CSV File Failure')

    def writeCSVRow(self, fileName, filePath, fileSize, mTime, aTime,
                    cTime, hashVal, own, grp, mod):
        self.writer.writerow((fileName, filePath, fileSize, mTime,
        aTime, cTime, hashVal, own, grp, mod))

    def writerClose(self):
        self.csvFile.close()

Writing ch03/_pfish.py


In [35]:
!python ch03/pfish.py --md5 -d './' -r './report' -v

Command line processed: Successfully
Welcome to p-fish ... version
Processing File: ./ch02.ipynb
Processing File: ./ch03.ipynb
Processing File: ./ch04.ipynb
Processing File: ./pFishLog.log
Processing File: ./.ipynb_checkpoints/ch02-checkpoint.ipynb
Processing File: ./.ipynb_checkpoints/ch03-checkpoint.ipynb
Processing File: ./.ipynb_checkpoints/ch04-checkpoint.ipynb
Processing File: ./ch03/_pfish.py
Processing File: ./ch03/_pfish.pyc
Processing File: ./ch03/pfish.py
Processing File: ./report/fileSystemReport.csv
Program End


In [36]:
!cat ./pFishLog.log

2015-10-03 00:55:10,845 
2015-10-03 00:55:10,845 Welcome to p-fish version 1.0 ... New Scan Started
2015-10-03 00:55:10,845 
2015-10-03 00:55:10,845 System: darwin
2015-10-03 00:55:10,846 Version: 2.7.10 (default, Jul 14 2015, 19:46:27) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.39)]
2015-10-03 00:55:10,846 Root Path: ./
2015-10-03 00:55:10,847 Files Processed:11
2015-10-03 00:55:10,847 Elapsed Time: 0.00207901000977seconds
2015-10-03 00:55:10,847 
2015-10-03 00:55:10,848 Program Terminated Normally
2015-10-03 00:55:10,848 


In [37]:
import pandas as pd

In [38]:
df = pd.read_csv('/tmp/fileSystemReport.csv')
df

,File,Path,Size,Modified Time,Access Time,Created Time,MD5,Owner,Group,Mode
0,ch02.ipynb,./ch02.ipynb,13006,Fri Oct 2 07:52:40 2015,Sat Oct 3 00:44:33 2015,Fri Oct 2 07:52:40 2015,597becf577c6a616c336c4e7660c0b9c,501,20,33152
1,ch03.ipynb,./ch03.ipynb,48386,Sat Oct 3 00:43:51 2015,Sat Oct 3 00:44:33 2015,Sat Oct 3 00:43:51 2015,966fa74c1c2b05fef9ee33b55ab95b2c,501,20,33152
2,pFishLog.log,./pFishLog.log,331,Sat Oct 3 00:44:33 2015,Sat Oct 3 00:44:33 2015,Sat Oct 3 00:44:33 2015,75c29e1b6243d6c1d0a8a5a239d8c3d0,501,20,33188
3,ch02-checkpoint.ipynb,./.ipynb_checkpoints/ch02-checkpoint.ipynb,13006,Fri Oct 2 07:52:40 2015,Sat Oct 3 00:44:33 2015,Fri Oct 2 07:52:40 2015,597becf577c6a616c336c4e7660c0b9c,501,20,33152
4,ch03-checkpoint.ipynb,./.ipynb_checkpoints/ch03-checkpoint.ipynb,33049,Fri Oct 2 14:12:46 2015,Sat Oct 3 00:44:33 2015,Fri Oct 2 14:12:46 2015,a31524f195333dc215d7e4a5851ec289,501,20,33152
5,_pfish.py,./ch03/_pfish.py,7991,Sat Oct 3 00:43:45 2015,Sat Oct 3 00:44:33 2015,Sat Oct 3 00:43:45 2015,c7b1c969f7105663bdb1fc67153129cd,501,20,33188
6,_pfish.pyc,./ch03/_pfish.pyc,6230,Sat Oct 3 00:44:09 2015,Sat Oct 3 00:44:33 2015,Sat Oct 3 00:44:09 2015,07596415f35c4323460164398c9ea842,501,20,33188
7,pfish.py,./ch03/pfish.py,1317,Sat Oct 3 00:43:22 2015,Sat Oct 3 00:44:33 2015,Sat Oct 3 00:43:22 2015,5998bff15b4cebde309b7c4dabaf9281,501,20,33188


In [39]:
!column ./report/fileSystemReport.csv

### 소스 후기

- Indent가 엉망이다
- 그리고 PEP8을 지키지 않아서 짜증 유발.. 왠 함수명에 대문자냐?!!
  - 명명법까지 보여주는 것은 좋으나 파이썬에서 쓰는 명명법이 아니기 때문에..
  - 명명법까지 명시해줄 정도면 저자가 꽤 친절한거다.
- 번역자가 영어만 번역한 것 같은 느낌이 강하게 든다. 코드를 하나하나 쳐봤다면 

```python
report error
```

- 라는걸 그냥 두지는 않았을텐데
- 초보자가 이 책을 보고 따라하기에는 무리가 있어 보인다.
- 왜 똑같이 했는데 안되지? 라는 생각이 강하게 들기 때문에 엄청 짜증날 것 같다.

## 복습

- 표준 라이브러리 활용
- 명령줄 구문분석
- 명령줄 매개변수 유효성 검사
- 파이썬 로거 활성화하고 행위에 대한 수사 기록을 제공하기 위한 로깅 시스템에 이벤트와 오류 보고
- 단방향 해시 알고리즘의 선택에 대한 기능과 처리된 각 파일의 주요 속성을 추출하는 기능
- csv 모듈 활용

## 요약 질문

1. 단방향 해시 알고리즘을 추가한다면 어떤 기능을 수정해야 할 것인가? 또한, 파이썬 표준 라이브러리 사용에 의해서만 어떤 다른 단방향 해시 알고리즘을 쉽게 이용할 수 있다.
2. 2개의 전역 변수의 필요성을 제거하고 싶다면 클래스 사용에 의해 쉽게 이를 달성할 수 있는 방법은 무엇인가? 어떤 기능이 클래스로 전환되어야 하고 어떤 메소드의 생성이 필요할까?
3. 어떤 이벤트 또는 요소들이 기록되어야 한다고 생각하는가? 여러분은 그 일을 어떻게 할 것인가?
4. 어떤 추가한 열을 보고서에 표시하고 추가 정보를 얻을 수 있는 방법은 무엇인가?
5. 로그에는 어떤 부가적인 정보(예를 들어, 수사관 이름 또는 사례 번호 같은)가 포함되어야 한다. 어떻게 그런 정보를 얻을 것인가?